Reliability Models for Facility Location:The Expected Failure Cost Case
===

Base on the following article:

*Snyder, L. V., & Daskin, M. S. (2005). Reliability models for facility location: the expected failure cost case. Transportation Science, 39(3), 400-416.*

Part III - The Reliability P-Median Problem
---

In [15]:
# Colecting the problem
import sys
sys.path.append('../../PythonLib')

from dataset.mongodb import MongoClient
# Solving the problem
from solvers.uflp import uflp
# Representing the network
from dataviz.network import Network

from sklearn.neighbors import DistanceMetric
from geopy.distance import great_circle

import scipy as sp
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First, we collect the dataset from our database using **MonClient**.

In [16]:
dataset = MongoClient(db = {
# MongoClient let us connect to the database and manipulate our dataset
    "mongo_host": "ns396089.ip-37-59-38.eu",
    "mongo_port": 32771,
    "mongo_db_name": "NETWORK-AND-DISCRETE-LOCATION"
}, q = {
    'metadata.author': 'Mark S. Daskin',
    'metadata.topic': 'NETWORK-AND-DISCRETE-LOCATION',
    'metadata.dataset': '49-nodes'
},f=None)

We can display the dataset using the method **get** which return a pandas Dataframe containing our data

In [28]:
df = dataset.get()
df.head()

df = df.append({'ID' : '' , 'CITY' : "Dummy", "STATE POP": sum(df["STATE POP"])} , ignore_index=True)

The reliable P-median objective aims to minimise of the weighted sum of the **operating cost** and the **expected failure cost**. Let consider a set of **Customers** ($I$), and a set of potential location we called **Facilities** ($J$). The set of **Facilities** ($J$) is devided in two, one is the set of **nonfailable facility** ($NF$) and the set of facilities that may fail ($F$), $NF \cup F = J$. Each *customer* node has an associated **demande** ($h_i$). The cost **shiping cost** ($d_{ij}$) represent to deliver a unit of demande from a *facility* to a *customer*. Each customer applies an **penalty** ($\theta_i$) for each unit of demande that is not serve. 

The **penalty** ($\theta_i$) is incure if all facilites have failled are if the value of that penalty is smaller than the **shiping cost** ($d_{ij}$) to any of the existing facilities. To model this, we add an **emergency facility** ($u \in NF$) that is nonfailiable and as a transportation cost equals to the **penalty**.

Each facility as an expected **failure pobability** ($q_j$).

In [59]:
#P-Median Problem
P=5
alpha = 0.5
dummy_id = 49



# Sets of Customers abd Facilities
I  = Customers  = df.index
J  = Facilities = df.index
F  = J[:-1]
NF = [J[-1]]

# Sphiping cost
## Demande are set to the state populatio divides by  10^5 for 49-nodes and 10^4 for the others
h = demande = df["STATE POP"] / 100000
## the transportation cost is set to the great circle distance between i and j
d = shiping_cost = DistanceMetric.get_metric('haversine').pairwise(df[["LATITUDE","LONGITUDE"]].apply(np.radians)) * 3959

# Penalty for not shiping (Nan value represente value for the dummy facility becaut it as not Longitude or latitude)
d[np.isnan(d)] = 10000

# Probability of failure
## the probability of faillure was set to 0.05
q = failure_probability = 0.05

Similarly to UFLP there is two set of decision variables: **locations variables** ($X$) and **assignements variables** ($Y$)
$$
X_j =\left\{
        \begin{array}{ll}
          1, \text{if a facility is open}\\
          0, \text{otherwise}\\
        \end{array}
      \right.
$$

$$
Y_{ijr} =\left\{
        \begin{array}{ll}
          1, \text{if demand node i is assignement to j as a level r-assignement}\\
          0, \text{otherwise}\\
        \end{array}
      \right.
$$

In [60]:
from docplex.mp.model import Model
###################
# create one model instance
m = Model(name="Reliable P-Median problem")

###################
# Define variables

# x(j) equals 1 if node j in the solution
X = m.binary_var_dict([(j) 
                      for j in J], name="X")
# y(j,j,r) equales 1 if node j is in the solution
Y = m.binary_var_dict([(i,j,r) 
                      for i in I
                      for j in J
                      for r in range(P-1)], name="Y")

The objective $w_1$ mesure the operation cost related to the P-median problem. The objective $w_2$ compute the expected cost of failure.

$$
w_1 = \sum_{i \in I} \sum_{j \in J} h_i d_{ij} Y_{ij0}
$$

$$
w_2 = \sum_{i \in I} h_i [ \sum_{j \in NF} \sum_{r=0}^{p-1} d_{ij} q^{r} Y_{ijr} + \sum_{j \in F} \sum_{r=0}^{p-1} d_{ij} (1-q) Y_{ijr} ]
$$

In [61]:
###################
# Define Objective
w1 = m.sum(h[i] * d[i][j] * Y[i,j,0] for i in I for j in J)
w2_NF = q * m.sum(h[i] * d[i][j] * Y[i,j,r] for j in NF for i in I for r in range(P-1))
w2_F  = (1 - q)  * m.sum(h[i] * d[i][j] * Y[i,j,r] for j in F for i in I for r in range(P-1))
w2 = w2_NF + w2_F

m.minimize(alpha * w1 + (1-alpha) * w2)

In [62]:
###################
# Define KPI
m.add_kpi(w1, "Operating cost")
m.add_kpi(w2, "Expected Failure cost")

DecisionKPI(name=Expected Failure cost,expr=702004.386Y_(0, 1, 0)+702004.386Y_(0, 1, 1)+702004.386Y_(0, 1, 2..)

In [65]:
###################
# Define Constraints

## 1
for i in I:
    for r in range(P-1):
        m.add_constraint(m.sum(Y[i,j,r] for j in J) + m.sum(Y[i,j,s] for j in NF for s in range(P-1)) == 1, ctname='assignement_%s_%s' % (i,r))
## 2
for i in I:
    for j in J:
        for r in range(P-1):
            m.add_constraint(Y[i,j,r] - X[j] <= 0, ctname='closed_facility_%s_%s_%s' % (i,j,r))
            
## 3
m.add_constraint(m.sum(X[j] for j in J) <= P, ctname='p_median')

## 4
for i in I:
    for j in J:
        m.add_constraint(m.sum(Y[i,j,r] for r in range(P-1)) == 1, ctname='single_lvl_assignement_%s_%s' % (i,j))
        
## 5
m.add_constraint(X[dummy_id] == 1, ctname='dummy_facility')

docplex.mp.LinearConstraint[dummy_facility](X_49,EQ,1)